# Chrome + AirPlay Test Harness Report

This report details the execution of automated tests for Chrome rendering and AirPlay casting functionality.

**Run Information:**
- **Timestamp (UTC):** `$(python -c 'import datetime; print(datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC"))')`
- **Current Git SHA:** `$(git rev-parse HEAD)`

In [ ]:
!ROOT_DIR="$(pwd)"
!echo "Current working directory for dashboard script: $ROOT_DIR"
!echo "Attempting to start dashboard if not already running..."
# Check if ports are in use
!if ! (lsof -i :3000 -t >/dev/null && lsof -i :8000 -t >/dev/null); then \
    echo "Dashboard not detected, starting with $ROOT_DIR/run_dashboard.sh ..."; \
    nohup "$ROOT_DIR/run_dashboard.sh" > dashboard_run.log 2>&1 & \
    echo "Waiting for dashboard to initialize (15 seconds)..."; \
    sleep 15; \
    echo "Dashboard startup initiated. Check dashboard_run.log for details."; \
else \
    echo "Dashboard appears to be running on ports 3000 and/or 8000."; \
fi

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import datetime
import time
import os

print(f"[{datetime.datetime.now()}] Starting Chrome test...")
url = "http://localhost:3000"
screenshot_filename = "chrome_screenshot.png"
screenshot_path = os.path.join(os.getcwd(), screenshot_filename)
console_errors_found = False

chrome_options = Options()
chrome_options.add_argument("--headless") # Run headless
chrome_options.add_argument("--no-sandbox") # Bypass OS security model, REQUIRED for Docker/CI
chrome_options.add_argument("--disable-dev-shm-usage") # Overcome limited resource problems
chrome_options.add_argument("--window-size=1280,720") # Specify window size

# Enable browser logging
dc = DesiredCapabilities.CHROME
dc['goog:loggingPrefs'] = { 'browser':'ALL' }

driver = None
try:
    print(f"[{datetime.datetime.now()}] Initializing Chrome WebDriver...")
    driver = webdriver.Chrome(options=chrome_options, desired_capabilities=dc)
    print(f"[{datetime.datetime.now()}] Navigating to {url}...")
    driver.get(url)
    
    # Wait for a known element to ensure page is loaded, e.g., the body tag or a specific div
    # Adjust timeout and element as necessary for your app
    print(f"[{datetime.datetime.now()}] Waiting for page to load...")
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    time.sleep(5) # Additional wait for dynamic content if any
    
    print(f"[{datetime.datetime.now()}] Capturing screenshot to {screenshot_path}...")
    driver.save_screenshot(screenshot_path)
    print(f"[{datetime.datetime.now()}] Screenshot saved.")

    print(f"[{datetime.datetime.now()}] Checking for console errors...")
    logs = driver.get_log('browser')
    for entry in logs:
        if entry['level'] == 'SEVERE':
            print(f"CONSOLE ERROR: {entry['message']}")
            console_errors_found = True
    
    if console_errors_found:
        print("FAIL: Severe console errors detected.")
        # Optionally, raise an exception to mark cell execution as failed
        # raise Exception("Severe console errors detected")
    else:
        print("SUCCESS: Page loaded and no severe console errors detected.")

except Exception as e:
    print(f"FAIL: An error occurred during Chrome test: {e}")
    # Optionally, raise e
finally:
    if driver:
        print(f"[{datetime.datetime.now()}] Closing Chrome WebDriver...")
        driver.quit()
        print(f"[{datetime.datetime.now()}] Chrome WebDriver closed.")

In [ ]:
!API_BASE_URL="http://localhost:8000/api"
!PROJECTOR_ID="proj-hccast" # Default AirPlay test projector from renderer_config.json
!SCENE_ID="overlay-frontdoor"  # Default scene for proj-hccast

!echo "Attempting to cast to AirPlay device: $PROJECTOR_ID with scene: $SCENE_ID via $API_BASE_URL/renderer/start"
!RESPONSE=$(curl -s -w "\n%{http_code}" -X POST \
  -H "Content-Type: application/json" \
  -d "{\
        \"projector\": \"$PROJECTOR_ID\",\
        \"scene\": \"$SCENE_ID\"\
      }" \
  "$API_BASE_URL/renderer/start")

# Extract body and status code
!BODY=$(echo "$RESPONSE" | sed '$d')
!STATUS_CODE=$(echo "$RESPONSE" | tail -n1)

!echo "Status Code: $STATUS_CODE"
!echo "Response Body: $BODY"

!if [ "$STATUS_CODE" -ge 200 ] && [ "$STATUS_CODE" -lt 300 ]; then \
  echo "AirPlay command API call successful (HTTP $STATUS_CODE)."; \
  exit 0; \
else \
  echo "AirPlay command API call failed (HTTP $STATUS_CODE)."; \
  exit 1; \
fi

## Test Summary

This section should be filled in after running the notebook cells above. 
It will summarize the outcomes of the Chrome rendering test and the AirPlay cast command.

*   **Page Rendered (Cell 3):** [✔ or ❌] [Details: e.g., Page loaded successfully. No console errors. / Console error detected: (details)]
    *   *Screenshot:* `chrome_screenshot.png` (if successful)
*   **AirPlay Command (Cell 4):** [✔ or ❌] [Details: e.g., AirPlay command API call successful (HTTP 200). / AirPlay command API call failed (HTTP XXX) with error: (details)]
    *   *Status Code:* [e.g., 200]
    *   *Response Body:* [e.g., `{"success":true,...}`]

In [ ]:
from IPython.display import Image, display
import os

screenshot_filename = "chrome_screenshot.png"
screenshot_path = os.path.join(os.getcwd(), screenshot_filename)

if os.path.exists(screenshot_path):
    print(f"Displaying screenshot from: {screenshot_path}")
    display(Image(filename=screenshot_path))
else:
    print(f"Screenshot file not found at {screenshot_path}. Was Cell 3 successful?")